In [1]:
# !pip install textblob
# !pip install gensim
# nltk.download('stopwords')

In [2]:
import re
import numpy as np
import pandas as pd
from textblob import Word
from nltk.corpus import stopwords

In [3]:
# Importing the training data

data_folder_path = "C:\\Users\\swapn\\OneDrive\\Desktop\\NLP\\Assignment2\\NLP_AUTUMN_ASSIGNMENT_DATA\\"

train_file_path = data_folder_path + "NLP_ass_train.tsv"
val_file_path = data_folder_path + "NLP_ass_valid.tsv"
test_file_path = data_folder_path + "NLP_ass_test.tsv"

train_df = pd.read_csv(train_file_path, delimiter='\t')
val_df = pd.read_csv(val_file_path, delimiter='\t')
test_df = pd.read_csv(test_file_path, delimiter='\t')


# Renaming the columns
df1 = pd.DataFrame({train_df.columns[0]: [train_df.columns[0]], train_df.columns[1]: [train_df.columns[1]]})
train_df = pd.concat([df1, train_df])
train_df = train_df.reset_index(drop=True)
train_df.columns = ["Text", "Label"]

df2 = pd.DataFrame({val_df.columns[0]: [val_df.columns[0]], val_df.columns[1]: [val_df.columns[1]]})
val_df = pd.concat([df2, val_df])
val_df = val_df.reset_index(drop=True)
val_df.columns = ["Text", "Label"]

df3 = pd.DataFrame({test_df.columns[0]: [test_df.columns[0]], test_df.columns[1]: [test_df.columns[1]]})
test_df = pd.concat([df3, test_df])
test_df = test_df.reset_index(drop=True)
test_df.columns = ["Text", "Label"]

In [4]:
train_df

,Text,Label
0,we cannot continue calling ourselves feminists...,normal
1,nawt yall niggers ignoring me,normal
2,<user> i am bit confused coz chinese ppl can n...,hatespeech
3,and this is why i end up with nigger trainee d...,hatespeech
4,nogs jews and dykes how enriching,offensive
...,...,...
15378,this vile racist ghetto ass dumber than a reta...,offensive
15379,when i first got on here and said i hate trump...,offensive
15380,was macht der moslem wenn der zion gegen seine...,normal
15381,it is awful look at world demographics asians ...,hatespeech


In [5]:
val_df

,Text,Label
0,this bitch in whataburger eating a burger with...,hatespeech
1,tired i can not support abortion from a moral ...,hatespeech
2,all of my exes were cute but they were hoes i ...,offensive
3,<user> <user> they visited helped provided foo...,normal
4,it funny how people offended by the word nigge...,normal
...,...,...
1917,why do you think all these various muslim grou...,offensive
1918,this is pretty much the conclusion of the fran...,hatespeech
1919,graph straftaten gegen die sexuelle selbstbest...,normal
1920,an afghani immigrant once told me that in afgh...,normal


In [6]:
test_df

,Text,Label
0,i dont think im getting my baby them white 9 h...,normal
1,laura loomer raped me while screaming at me in...,hatespeech
2,<user> what did the old lady do was she a nazi...,normal
3,as much as i appreciate a format of open discu...,normal
4,sex be so good a bitch be slow stroking and cr...,offensive
...,...,...
1919,it always women trying this shit like dingbat ...,offensive
1920,because women would never lie about being sexu...,offensive
1921,how is all that awesome muslim diversity going...,offensive
1922,well my dear lgbtq brothers and sisters i do n...,hatespeech


In [7]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove special characters and symbols
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Handling Negations
    # Detect and handle negations by appending "not_" to the words that follow negation terms.
    words = text.split()
    for i in range(len(words)):
        if words[i] == 'not' and i < len(words) - 1:
            words[i + 1] = 'not_' + words[i + 1]
    
    text = ' '.join(words)
    
    # Tokenize the text (split it into words)
    words = text.split()
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    text = ' '.join(words)
    
    # Token Normalization
    # Normalize words by reducing them to their base forms.
    words = text.split()
    normalized_words = [Word(word).lemmatize() for word in words]
    
    # Join the words back into a cleaned text
    cleaned_text = ' '.join(normalized_words)
    return cleaned_text


In [8]:
# Preprocessing the data
train_df['Text'] = train_df['Text'].apply(preprocess_text)
val_df['Text'] = val_df['Text'].apply(preprocess_text)
test_df['Text'] = test_df['Text'].apply(preprocess_text)

In [9]:
train_df

,Text,Label
0,cannot continue calling feminist right womxn a...,normal
1,nawt yall nigger ignoring,normal
2,user bit confused coz chinese ppl not_access t...,hatespeech
3,end nigger trainee doctor not_speak properly l...,hatespeech
4,nog jew dyke enriching,offensive
...,...,...
15378,vile racist ghetto as dumber retarded tree sna...,offensive
15379,first got said hate trump considered troll peo...,offensive
15380,macht der moslem wenn der zion gegen seinen pr...,normal
15381,awful look world demographic asian fucking eve...,hatespeech


In [10]:
val_df

,Text,Label
0,bitch whataburger eating burger top bun holdin...,hatespeech
1,tired not_support abortion moral standpoint wi...,hatespeech
2,ex cute hoe guess attract fly looking thots,offensive
3,user user visited helped provided food kp isla...,normal
4,funny people offended word nigger problem call...,normal
...,...,...
1917,think various muslim group middle east india a...,offensive
1918,pretty much conclusion frankfurt school said o...,hatespeech
1919,graph straftaten gegen die sexuelle selbstbest...,normal
1920,afghani immigrant told afghanistan not_believe...,normal


In [11]:
test_df

,Text,Label
0,dont think im getting baby white 9 two white j...,normal
1,laura loomer raped screaming disgusting kike l...,hatespeech
2,user old lady nazi white supremacist,normal
3,much appreciate format open discussion really ...,normal
4,sex good bitch slow stroking cry,offensive
...,...,...
1919,always woman trying shit like dingbat dingell ...,offensive
1920,woman would never lie sexually assaulted perso...,offensive
1921,awesome muslim diversity going native german a...,offensive
1922,well dear lgbtq brother sister not_want die ha...,hatespeech


In [12]:
from sklearn.preprocessing import LabelEncoder

X_train = train_df[train_df.columns[0]]
X_val = val_df[val_df.columns[0]]
X_test = test_df[test_df.columns[0]]
y_train = train_df[train_df.columns[1]]
y_val = val_df[val_df.columns[1]]
y_test = test_df[test_df.columns[1]]

# Encode the labels using LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.fit_transform(y_val)
y_test = label_encoder.fit_transform(y_test)

In [13]:
from gensim.models import KeyedVectors

# Load the Google Word2Vec model in binary format
# I set the limit as it is unable to load the entire model
limit = 1000000 # For 8GB RAM
word2vec_model_path = data_folder_path[:-27] + "GoogleNews-vectors-negative300.bin\\GoogleNews-vectors-negative300.bin"
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=True, limit=limit)

In [14]:
# Convert text to Word2Vec embeddings
def document_embedding(text, model):
    words = text.split()
    word_embeddings = [model[word] for word in words if word in model]
    if word_embeddings:
        doc_embedding = np.mean(word_embeddings, axis=0)  # Average the word embeddings
    else:
        doc_embedding = np.zeros(model.vector_size)  # Use zero vectors for out-of-vocabulary words
    return doc_embedding

In [15]:
# Apply the document_embedding function to your dataset
X_train = train_df[train_df.columns[0]].apply(lambda x: document_embedding(x, word2vec_model))
X_train = np.stack(X_train)  # Convert the list of embeddings to a NumPy array

X_val = val_df[val_df.columns[0]].apply(lambda x: document_embedding(x, word2vec_model))
X_val = np.stack(X_val)  # Convert the list of embeddings to a NumPy array

X_test = test_df[test_df.columns[0]].apply(lambda x: document_embedding(x, word2vec_model))
X_test = np.stack(X_test)  # Convert the list of embeddings to a NumPy array

In [16]:
# Creating the model

from tensorflow import keras

# Create a Neural Network model
model = keras.Sequential()
model.add(keras.layers.Dense(128, activation='relu', input_shape=(word2vec_model.vector_size,)))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))  # Use 'softmax' activation for multi-class classification

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert labels to one-hot encoding
num_classes = 3  # Number of classes
y_train_one_hot = keras.utils.to_categorical(y_train, num_classes)
y_val_one_hot = keras.utils.to_categorical(y_val, num_classes)

In [17]:
# Setting up Early Stopping
from tensorflow.keras.callbacks import EarlyStopping

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [18]:
# Training and Saving the model
# Model with early stopping gives us relatively better results

epochs = 100
batch_size = 64

# Train the model with early stopping
history = model.fit(X_train, y_train_one_hot, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val_one_hot), callbacks=[early_stopping])

# Train the model without early stopping
# model.fit(X_train, y_train_one_hot, epochs=50, validation_data=(X_val, y_val_one_hot))

# Save the trained model to a file
model.save('Neural_Network_Model.h5')


Epoch 1/100
241/241 [==============================] - 1s 3ms/step - loss: 0.9164 - accuracy: 0.5754 - val_loss: 0.8880 - val_accuracy: 0.6160
Epoch 2/100
241/241 [==============================] - 0s 2ms/step - loss: 0.8407 - accuracy: 0.6241 - val_loss: 0.8685 - val_accuracy: 0.6155
Epoch 3/100
241/241 [==============================] - 0s 2ms/step - loss: 0.8155 - accuracy: 0.6377 - val_loss: 0.8513 - val_accuracy: 0.6217
Epoch 4/100
241/241 [==============================] - 0s 2ms/step - loss: 0.7934 - accuracy: 0.6503 - val_loss: 0.8555 - val_accuracy: 0.6259
Epoch 5/100
241/241 [==============================] - 0s 2ms/step - loss: 0.7737 - accuracy: 0.6614 - val_loss: 0.8551 - val_accuracy: 0.6160
Epoch 6/100
241/241 [==============================] - 0s 2ms/step - loss: 0.7532 - accuracy: 0.6708 - val_loss: 0.8533 - val_accuracy: 0.6124


C:\Users\swapn\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
# Testing the saved model

# Load the saved model
model_path = data_folder_path[:-27] + "Neural_Network_Model.h5"
loaded_model = keras.models.load_model(model_path)

# Make predictions on the test data
predictions = loaded_model.predict(X_test)

# The 'predictions' variable will contain the model's output for each data point in X_test.
from sklearn.metrics import accuracy_score, classification_report

# Convert the predicted probabilities to class labels (0, 1, 2)
predicted_labels = predictions.argmax(axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)

# Generate a classification report for more detailed metrics
report = classification_report(y_test, predicted_labels)

print("Accuracy:", f'{100*accuracy}'+"%")
print("Classification Report:\n", report)

61/61 [==============================] - 0s 569us/step
Accuracy: 62.00623700623701%
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.69      0.67       594
           1       0.64      0.73      0.68       782
           2       0.54      0.38      0.45       548

    accuracy                           0.62      1924
   macro avg       0.61      0.60      0.60      1924
weighted avg       0.61      0.62      0.61      1924



In [20]:
# Accuracy: 62.16216216216216%
# Classification Report:
#                precision    recall  f1-score   support

#            0       0.69      0.66      0.67       594
#            1       0.61      0.76      0.68       782
#            2       0.54      0.39      0.45       548

#     accuracy                           0.62      1924
#    macro avg       0.62      0.60      0.60      1924
# weighted avg       0.62      0.62      0.61      1924

In [21]:
def calculate_intersection(set1, set2):
    return len(set(set1) & set(set2))

# Example train, validation, and test sets (replace with your actual data)
train_set = set(train_df["Text"])
validation_set = set(val_df["Text"])
test_set = set(test_df["Text"])

# Calculate and print the intersections
intersection_train_validation = calculate_intersection(train_set, validation_set)
intersection_train_test = calculate_intersection(train_set, test_set)
intersection_validation_test = calculate_intersection(validation_set, test_set)

print("Intersection between train and validation sets:", intersection_train_validation)
print("Intersection between train and test sets:", intersection_train_test)
print("Intersection between validation and test sets:", intersection_validation_test)

Intersection between train and validation sets: 9
Intersection between train and test sets: 14
Intersection between validation and test sets: 2


In [ ]:
# Intersection between train and validation sets: 9
# Intersection between train and test sets: 14
# Intersection between validation and test sets: 2